# Data Exploration

In [ ]:
import sys
sys.path.append('src')
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import inspect
import matplotlib.pyplot as plt
import os.path as osp
import pickle
from sklearn.model_selection import train_test_split
# Local Modules
from utils import make_st_map_interactive
from data_funcs import train_test_split_spacetime
import reproducibility

## Cross-Validation Setup 

To get reliable estimates of forecast error of a spatiotemporal model, care must be taken to avoid data leakage. See: https://github.com/jh-206/FRAMSC-2024---FMDA-Data-and-CV-Methods/blob/main/Spatiotemporal%20Cross%20Validation.ipynb

In [ ]:
df = pd.read_pickle("data/raws_df0.pkl")

In [ ]:
# make_st_map_interactive(df)

In [ ]:
print(f"Total Locations: {len(df.STID.unique())}")
print(f"Earliest Time: {df.index.min()}")
print(f"Latest Time: {df.index.max()}")
df.head()

## Broken Sensor Data Filters

Some RAWS fuel moisture sensors are obviously faulty. Such as below,

In [ ]:
st = "SAWC2"
month = 2
year = 2024
df_temp = df[(df.index.month == month) & (df.index.year == year)]
plt.plot(df_temp[df_temp.STID == st]['fm'])
plt.xticks(rotation=90, fontsize=8)
plt.show()

In order to flag periods of data such as above, we will remove all observations for a station if the number of hours where the fuel moisture content is identical to the previous hour is greater than 24. So this would mean there is at least 1 full day of data that is suspect. 

In [ ]:
# Function given station and time period, return if first difference of FM is zero for greater than 24 hours of observations
def detect_bad_data(df0):
    lags = np.diff(df_temp[df_temp.STID == "SAWC2"]['fm'], n=1)
    zero_lags = np.count_nonzero(lags == 0)
    return zero_lags > 24

In [ ]:
month_year = df.index.to_period('M').unique()
flags = []
for my in month_year:
    print("~"*50)
    month = my.month
    year = my.year
    for st in df.STID.unique():
        print(f"Detecting bad data for month: {my}, and Station: {st}")
        df_temp = df[(df.index.month == month) & (df.index.year == year) & (df.STID == st)]
        bad = detect_bad_data(df_temp)
        if bad:
            print(bad)
            flags.append({'STID': st,'my': my})

In [ ]:
flags

All of the flagged periods were from the same station, so we will just drop this one entirely.

In [ ]:
# remove bad station from data
df = df[df['STID'] != 'SAWC2']

# Write Dataframe
with open(osp.join("data", "raws_df.pkl"), 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Summary Stats

In [ ]:
print(f"Min Date: {df.date.min()}")
print(f"Max Date: {df.date.max()}")

In [ ]:
df.shape

In [ ]:
cols_list = ["Ed", "rain", "wind", "solar", "hour", "doy", "lat", "lon", "elev"]
summary_df = df[cols_list].agg(['min', 'max', 'mean']).transpose()
summary_df = summary_df.round(2)
summary_df = summary_df.map(lambda x: '{:.0f}'.format(x) if x.is_integer() else '{:.2f}'.format(x))
summary_df

In [ ]:
df = df.dropna()

In [ ]:
print(f"Data Shape: {df.shape}")

In [ ]:
print(f"Unique Locations: {len(df.STID.unique())}")

In [ ]:
print(summary_df.to_latex())

## Spatiotemporal CV

For a meaningful analysis of forecast error for a spatiotemporal model, the test set must consist of locations that were not included in the training and at times in the future of training. To conduct this split, we use a custom function `train_test_split_spacetime`, that mimics the return format of the typicaly `sklearn` function `train_test_split`, while accounting for relationships in space and time.

In [ ]:
# Print function
print(inspect.getsource(train_test_split_spacetime))

In [ ]:
reproducibility.set_seed(42)
X_train, X_test, y_train, y_test = train_test_split_spacetime(
    df, 
    test_days = 2,
    spatial_test_frac = 0.2,
    verbose = True
)

## Visualize Method

Rolling window of 30 days, 28 train 2 test. Should match output of loop in notebook `2_loss.ipynb`.

Plot based on this source: https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4

## Exploratory Plots

### FMC Plots

In [ ]:
dat = df[df.STID == "CHAC2"]
times = np.arange(1600, 1750)
fig, ax1 = plt.subplots()
line1, = ax1.plot(dat.index[times], dat.fm.iloc[times], linestyle='-',c='#8BC084',label='FM Observed')
ax1.set_xlabel("Time")
ax1.grid()
ax1.set_ylim(0, 12)
ax1.set_ylabel("FM (%)")
ax1twin = ax1.twinx()
ax1.tick_params(axis='x', rotation=90)
line2, = ax1twin.plot(dat.rain.iloc[times], c='b',label='Rain', alpha=.8)
ax1twin.set_ylabel('Rain (mm/hr)')
ax1twin.set_ylim(0, 8)
ax1.set_title(f"RAWS Station CHAC2 - No Rain")

lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper left', bbox_to_anchor=(1.05, .95), fontsize='small')
plt.tight_layout()
plt.savefig('outputs/no_rain_plot.png')
plt.show()

In [ ]:
dat = df[df.STID == "CHAC2"]
times = np.arange(1600, 1820)
fig, ax1 = plt.subplots(figsize=(8, 4))
line1, = ax1.plot(dat.index[times], dat.fm.iloc[times], linestyle='-',c='#8BC084',label='FM Observed')
ax1.set_xlabel("Time")
ax1.grid()
ax1.set_ylim(0, 27)
ax1.set_ylabel("FM (%)")
ax1twin = ax1.twinx()
ax1.tick_params(axis='x', rotation=90)
line2, = ax1twin.plot(dat.rain.iloc[times], c='b',label='Rain', alpha=.8)
ax1twin.set_ylabel('Rain (mm/hr)')
ax1twin.set_ylim(0, 8)
ax1.set_title(f"RAWS Station CHAC2 - No Rain")

lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper left', bbox_to_anchor=(1.05, .95), fontsize='small')
plt.tight_layout()
plt.savefig('outputs/rain_plot.png')
plt.show()

In [ ]:
# dat = df[df.STID == "CHAC2"]
plt.figure(figsize=(6, 4))
times = np.arange(1600, 1750)
plt.plot(dat.fm.iloc[times], linestyle='-',c='#8BC084',label='FM Observed')
plt.plot(dat.rain.iloc[times], c='b',label='Rain', alpha=.8)
plt.plot(dat.Ew.iloc[times],label='Wetting Equilibrium', linestyle='--',c='#7CCCEF', alpha=.8)
plt.plot(dat.Ed.iloc[times],label='Drying Equilibrium', linestyle='--',c='#EF847C', alpha=.8)
plt.ylabel("FM (%)")
plt.xlabel("Time")
plt.ylim(0, 14)
plt.xticks(rotation=90, fontsize=8)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize='small')
plt.title("RAWS Station CHAC2 - No Rain")
plt.grid()
plt.tight_layout()
plt.savefig('outputs/eq_plot.png')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
hours = 1000
ax1.scatter(dat['rh'][0:hours], dat['fm'][0:hours], alpha=.8)
ax1.set_ylabel("Fuel Moisture (%)")
ax1.set_xlabel("Relative Humidity (%)")
ax2.scatter(dat['temp'][0:hours], dat['fm'][0:hours], alpha=.8)
ax2.set_xlabel("Temp (deg K)")
# plt.savefig('outputs/rh_temp_plot.png')

In [ ]:
plt.scatter(dat['rh'][0:hours], dat['temp'][0:hours])

In [ ]:
plt.scatter(dat['temp'][0:hours], dat['rh'][0:hours])

## Map Stations

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import random

In [ ]:
df['color'] = np.random.choice(['red', 'blue'], size=len(df))

In [ ]:

def make_map(df, box, save=None):
    fig = px.scatter_mapbox(df, 
                        lat="lat", 
                        lon="lon", 
                        color="color",
                        zoom=6.5, 
                        height=1800,
                        width=2400)

    fig.add_trace(go.Scattermapbox(
        mode="lines",
        lon=[box[0], box[2], box[2], box[0], box[0]],
        lat=[box[1], box[1], box[3], box[3], box[1]],
        marker=dict(size=8, color="black"),
        line=dict(width=1, color="black"),
        showlegend=False
    ))
    
    center_lon=df['lon'].median()*.972
    center_lat=df['lat'].median()*1.02
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_center=dict(lat=center_lat, lon=center_lon)
    )
    fig.update_traces(marker=dict(size=22), showlegend=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    if save is not None:      
        fig.write_image(save)
        
    config = {
            "toImageButtonOptions": {
                "format": "png",  # Can also use "jpeg", "svg", etc.
                "height": 1800,   # Image height in pixels
                "width": 2400,    # Image width in pixels
                "scale": 4.17     # Increase resolution to approximate 300 DPI
            }
        } 
    fig.show(config=config)
    # return fig



In [ ]:
df2 = df.drop_duplicates(subset = 'STID')
df2.shape

In [ ]:
bbox = [-111, 37, -95, 46]

In [ ]:
make_map(df2, bbox, save="outputs/cv_plot.png")